In [1]:
import numpy as np
import os,sys
sys.path.append('..')
from trainer import Trainer
from config import get_config
from utils import *
import pickle
import matplotlib.pyplot as plt

#from re_main import main
from trainer import *

from pprint import pprint
from models import *

/home/slcf/.pyenv/versions/3.6.2/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [3]:
print(x_train.shape)
X = tf.constant(x_train)
y = tf.constant(y_train)
print(X)

(60000, 28, 28)
Tensor("Const:0", shape=(60000, 28, 28), dtype=uint8)


In [4]:
image_batch, label_batch = tf.train.shuffle_batch(
[X,y],
batch_size = 32,
num_threads = 4,
capacity = 50000,
min_after_dequeue = 10000)

In [5]:
print(image_batch)

Tensor("shuffle_batch:0", shape=(32, 60000, 28, 28), dtype=uint8)


In [6]:
sess = tf.Session()


In [7]:
tf.train.start_queue_runners(sess)

[<Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 140504788350720)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 140504779958016)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 140504771565312)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 140504305055488)>]

In [9]:
print(sess.run(image_batch))

[[[[0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   ..., 
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]]

  [[0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   ..., 
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]]

  [[0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   ..., 
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]]

  ..., 
  [[0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   ..., 
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]]

  [[0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   ..., 
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]]

  [[0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   ..., 
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]]]


 [[[0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   ..., 
   [0 0 0 ..., 0 0 0]
   [0 0 0 ..., 0 0 0]
   [0 0 0 ...

In [3]:
data_loader = batch_generator(x_train,128,"gray")

In [4]:
x = data_loader
x = norm_img(x,"gray")
crrX = corrupt(x,"gaussian",0.3)
temp_num =x.get_shape().as_list()

In [5]:
x = tf.reshape(x,[-1,784])
rx,z,AE_var = AE_dense(crrX,[784,1024,1024]) # 내부적으로 flatten()

In [6]:
cost = -tf.reduce_mean(x*tf.log(rx+1e-10)+(1-x)*tf.log(1-rx+1e-10))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [7]:
from tqdm import trange
steps = tf.Variable(0,name = 'step', trainable = False)
sess = tf.Session()
tf.train.start_queue_runners(sess)

[<Thread(QueueRunnerThread-training_data/random_shuffle_queue-training_data/random_shuffle_queue_EnqueueMany, started daemon 140349255182080)>,
 <Thread(QueueRunnerThread-training_data/random_shuffle_queue-training_data/random_shuffle_queue_EnqueueMany, started daemon 140349246789376)>,
 <Thread(QueueRunnerThread-training_data/random_shuffle_queue-training_data/random_shuffle_queue_EnqueueMany, started daemon 140349238396672)>,
 <Thread(QueueRunnerThread-training_data/random_shuffle_queue-training_data/random_shuffle_queue_EnqueueMany, started daemon 140349230003968)>]

In [8]:
lr = tf.Variable(0.001,name='lr')
summary_op = tf.summary.merge([tf.summary.scalar("loss",cost),tf.summary.scalar("lr",lr)])

In [9]:
sess.run(tf.global_variables_initializer())

In [10]:
print(sess.run(lr))

0.001


In [11]:
print(sess.run(data_loader))

[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 ..., 
 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  

In [12]:
for step in trange(0,10000):
    
    fetch_dict = {"loss":cost}
    if step % 10 ==0:
        fetch_dict.update({
            "summary": summary_op,
            "loss":cost,
            "lr":lr
        })
    #print(step)
    result = sess.run(fetch_dict)
    if step % 10 ==0:
        loss = result['loss']
        print("[{}/{}] Loss: {:.6f}".format(step,100,loss))
    

  0%|          | 40/10000 [00:00<03:24, 48.65it/s] 

[0/100] Loss: 0.821374
[10/100] Loss: 0.822247
[20/100] Loss: 0.820266
[30/100] Loss: 0.819577
[40/100] Loss: 0.819825
[50/100] Loss: 0.820159
[60/100] Loss: 0.821600
[70/100] Loss: 0.819915
[80/100] Loss: 0.820238


  2%|▏         | 184/10000 [00:01<01:00, 163.18it/s]

[90/100] Loss: 0.819169
[100/100] Loss: 0.819593
[110/100] Loss: 0.819779
[120/100] Loss: 0.822228
[130/100] Loss: 0.819899
[140/100] Loss: 0.821140
[150/100] Loss: 0.819743
[160/100] Loss: 0.820188
[170/100] Loss: 0.821046
[180/100] Loss: 0.819538


  3%|▎         | 287/10000 [00:01<00:44, 215.87it/s]

...

[9690/100] Loss: 0.819334
[9700/100] Loss: 0.820217
[9710/100] Loss: 0.819769
[9720/100] Loss: 0.819186
[9730/100] Loss: 0.821048
[9740/100] Loss: 0.822176
[9750/100] Loss: 0.820118
[9760/100] Loss: 0.820901
[9770/100] Loss: 0.820400
[9780/100] Loss: 0.820557
[9790/100] Loss: 0.821005


 99%|█████████▊| 9856/10000 [00:20<00:00, 490.30it/s]

[9800/100] Loss: 0.820295
[9810/100] Loss: 0.817622
[9820/100] Loss: 0.818413
[9830/100] Loss: 0.820915
[9840/100] Loss: 0.821591
[9850/100] Loss: 0.820828
[9860/100] Loss: 0.820039
[9870/100] Loss: 0.819748
[9880/100] Loss: 0.819922
[9890/100] Loss: 0.820145
[9900/100] Loss: 0.820127


100%|██████████| 10000/10000 [00:20<00:00, 490.67it/s]

[9910/100] Loss: 0.819348
[9920/100] Loss: 0.820234
[9930/100] Loss: 0.822653
[9940/100] Loss: 0.819717
[9950/100] Loss: 0.819049
[9960/100] Loss: 0.821781
[9970/100] Loss: 0.820070
[9980/100] Loss: 0.821864
[9990/100] Loss: 0.818881


In [ ]:
x = tf.reshape(x,[-1,self.])

In [ ]:
with open('logs/bedroom_lsun_1120_134241/params.json') as f:
    aa = json.load(f)

In [ ]:
print(aa)
aa['is_train']=False
aa['data_path']

In [ ]:
data_loader = get_loader(aa['data_path'], aa['batch_size'], aa['input_scale_size'], aa['data_format'], aa['split']

In [ ]:
import argparse
print(aa.keys())
aa2 = argparse.Namespace()
for dk in aa.keys():
    exec("aa2.%s = aa[dk]" % (dk))

In [ ]:
asd = Trainer(aa2,data_loader)

In [ ]:
z_fixed = np.random.uniform(-1,1,size = (asd.batch_size,asd.z_num))

In [ ]:
c_fixed = np.zeros(shape = (asd.batch_size,asd.c_num*10),dtype=np.float32)
c_fixed[:,0] = 1
zc_fixed = np.concatenate([z_fixed, c_fixed], axis=1)
x_fake = asd.sess.run(asd.G,{asd.zc:zc_fixed})/255.
print(x_fake.shape)

In [ ]:
fig = plt.figure(figsize=(20,5))
for i in range(asd.batch_size):
    ax = fig.add_subplot(2, asd.batch_size/2 , i+1)
    plt.imshow(x_fake[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
c_fixed = np.zeros(shape = (asd.batch_size,asd.c_num*10),dtype=np.float32)
c_fixed[:,1] = 1
zc_fixed = np.concatenate([z_fixed, c_fixed], axis=1)
x_fake = asd.sess.run(asd.G,{asd.zc:zc_fixed})/255.
print(x_fake.shape)